In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.preprocessing import image
import numpy as np

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Set the paths to your dataset
train_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/train'
test_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/test'
val_dir = '/kaggle/input/chest-xray-pneumonia/chest_xray/val'

# Image dimensions and batch size
image_size = (224, 224)
batch_size = 32

In [3]:
# Data augmentation for the training dataset
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Preprocess and augment the training data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)

Found 5216 images belonging to 2 classes.


In [4]:
# Preprocess the test
test_datagen = ImageDataGenerator(rescale=1.0/255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)

Found 624 images belonging to 2 classes.


In [5]:
# Preprocess the validation data
val_generator = test_datagen.flow_from_directory(
    val_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary'
)

Found 16 images belonging to 2 classes.


In [6]:
#Pretrained VGG16 + CNN Model

base_model = VGG16(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False

x = layers.Flatten()(base_model.output)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(1, activation='sigmoid')(x)

model = Model(base_model.input, x)

58889256/58889256 [==============================] - 3s 0us/step


In [7]:
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_generator, epochs=5, validation_data=val_generator)

Epoch 1/5
163/163 [==============================] - 140s 803ms/step - loss: 0.5241 - accuracy: 0.8484 - val_loss: 0.4778 - val_accuracy: 0.7500
Epoch 2/5
163/163 [==============================] - 96s 587ms/step - loss: 0.2337 - accuracy: 0.9020 - val_loss: 0.7150 - val_accuracy: 0.6875
Epoch 3/5
163/163 [==============================] - 96s 587ms/step - loss: 0.2090 - accuracy: 0.9137 - val_loss: 0.5912 - val_accuracy: 0.6875
Epoch 4/5
163/163 [==============================] - 96s 586ms/step - loss: 0.1952 - accuracy: 0.9199 - val_loss: 0.4236 - val_accuracy: 0.7500
Epoch 5/5
163/163 [==============================] - 96s 590ms/step - loss: 0.1931 - accuracy: 0.9216 - val_loss: 0.4695 - val_accuracy: 0.6875


In [8]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test accuracy: {test_accuracy * 100:.2f}%')

20/20 [==============================] - 10s 485ms/step - loss: 0.2708 - accuracy: 0.9006
Test accuracy: 90.06%


In [9]:
# Save the model
model.save('/kaggle/working/pneumonia_model.h5')

In [15]:
# Load the trained model
model = tf.keras.models.load_model('/kaggle/working/pneumonia_model.h5')

# Load an example image for prediction
image_path = '/kaggle/input/chest-xray-pneumonia/chest_xray/val/NORMAL/NORMAL2-IM-1440-0001.jpeg'
img = image.load_img(image_path, target_size=(224, 224))
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)

# Make prediction
predictions = model.predict(img)
print(predictions)
# Interpret the prediction
if predictions[0] > 0.5:
    print("The image is NORMAL.")
else:
    print("The image indicates PNEUMONIA.")

1/1 [==============================] - 0s 151ms/step
[[0.]]
The image indicates PNEUMONIA.


In [16]:
# Load the trained model
model = tf.keras.models.load_model('/kaggle/working/pneumonia_model.h5')

# Load an example image for prediction
image_path = '/kaggle/input/chest-xray-pneumonia/chest_xray/val/PNEUMONIA/person1949_bacteria_4880.jpeg'
img = image.load_img(image_path, target_size=(224, 224))
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)

# Make prediction
predictions = model.predict(img)
print(predictions)
# Interpret the prediction
if predictions[0] > 0.5:
    print("The image is NORMAL.")
else:
    print("The image indicates PNEUMONIA.")

1/1 [==============================] - 0s 148ms/step
[[0.]]
The image indicates PNEUMONIA.
